In [22]:
import os
import xarray
import re
import pandas as pd
import json
import tempfile
import boto3
import rasterio
from datetime import datetime
from dateutil.relativedelta import relativedelta

%env AWS_PROFILE='ghgc-smce-mfa'

env: AWS_PROFILE='veda-smce-mfa'


In [23]:
session = boto3.Session(profile_name='veda-smce-mfa')
s3_client = session.client('s3')
raster_io_session = rasterio.env.Env(profile_name='veda-smce-mfa')

source_bucket_name = 'ghgc-data-store-dev'
target_bucket_name = ''
new_cog_folder = 'updated_with_nodata'
collection_name = 'eccodarwin-co2flux-monthgrid-v5'
prefix = 'ecco_darwin/'

In [24]:
def get_all_s3_keys(bucket, prefic):
    """Get a list of all keys in an S3 bucket."""
    keys = []

    kwargs = {"Bucket": bucket, "Prefix": prefix}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            if obj["Key"].endswith(".tif"):
                keys.append(obj["Key"])

        try:
            kwargs["ContinuationToken"] = resp["NextContinuationToken"]
        except KeyError:
            break

    return keys

In [25]:
keys = get_all_s3_keys(bucket_name, prefix)
keys[:5]
keys[0].split('/')[-1]

'TopDownEmissions_GOSAT_post_coal_GEOS_CHEM_2019.tif'

In [26]:
with raster_io_session:
    for key in keys:
        with rasterio.open(f's3://{bucket_name}/{key}') as src:
    # Read the data
            data = src.read()
            flat_arr = data.squeeze().flatten()

            # Count occurrences of each unique value
            unique_values, counts = np.unique(flat_arr, return_counts=True)

            # Find the index of the maximum count
            max_count_index = np.argmax(counts)

            # Get the value with the maximum occurrence
            value_with_max_occurrence = unique_values[max_count_index]
            
            # Get the metadata of the source file
            meta = src.meta.copy()

            # Update the metadata with the new "no data" value
            meta.update(nodata=-9999)

            # Replace original "no data" values in data with new "no data" value
            data[data == value_with_max_occurrence] = -9999

# Write the updated data to a new file
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file_path = temp_file.name

            # Write the updated data to the temporary file
            with rasterio.open(temp_file_path, "w", **meta) as dst:
                dst.write(data)

            # Upload the temporary file to S3
            s3_client.upload_file(
                Filename=temp_file_path,
                Bucket=bucket_name,
                Key=f"{new_cog_folder}/{collection_name}/{key.split('/')[1]}",
            )

            # Clean up the temporary file
            os.remove(temp_file_path)

S3UploadFailedError: Failed to upload /var/folders/7b/5rrvrjx51l54jchgs0tqps0c0000gn/T/tmp4248qoat to ghgc-data-store-dev/updated_with_nodata/gosat-based-ch4budget-yeargrid-v1/TopDownEmissions_GOSAT_post_coal_GEOS_CHEM_2019.tif: An error occurred (ExpiredToken) when calling the PutObject operation: The provided token has expired.